In [8]:
import os
import random
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
from torch_geometric.transforms import NormalizeFeatures

from models import GAT, GraphSAGE, GIN
from utils import train_model, test_model, train_constrative_model, valid_model
from mean_average_distance import MAD, MADGap
from virtualnode import VirtualClassNode, UnidirectionalVirtualClassNode

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [9]:
SAVE_PATH = 'results'
EARLY_STOPPING = 30

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device
dataset = CitationFull(root='dataset/Cora', name='Cora', transform=NormalizeFeatures())

In [10]:
data = dataset[0]
df = pd.DataFrame(data.x)
df['y'] = data.y
train, valid = train_test_split(df, stratify=df.y, test_size=0.4)
valid, test = train_test_split(valid, stratify=valid.y, test_size=0.5)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True
data.valid_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.valid_mask[valid.index]=True
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask[test.index]=True

In [11]:
mad = MAD(device=device, global_flag=True)
madgap = MADGap(device, 3, 8)

In [12]:
hyperparameters = {
    'virtualnode' : [UnidirectionalVirtualClassNode(), VirtualClassNode()],
    'temperature' : np.linspace(0.1, 1, num=10),
    'constrative_coef' : np.logspace(-4, -1, 6),
    'lr': np.logspace(-4, -2, 5)
}

tuning_result = pd.DataFrame({
                            'model' : [],
                            'virtualnode' : [],
                            'temperature' : [],
                            'constrative coef' : [],
                            'lr' : [],
                            'train_acc' : [],
                            'train_loss' : [],
                            'val_acc' : [],
                            'val_loss' : [],
                            'test_acc' : [],
                            'macro f1' : [],
                            'micro f1'
                            'minor f1' : [],
                            'mad' : [],
                            'madgap' : []                            
                            })


In [14]:
for virtualnode in hyperparameters['virtualnode']:
    if virtualnode is None:
        vc = 'None'
        data_for_tuning = data
        constrative_flag = False
    else:
        vc = virtualnode
        data_for_tuning = vc.forward(data)
        constrative_flag = True
        
    for temperature in hyperparameters['temperature']:
        for constrative_coef in hyperparameters['constrative_coef']:
            for lr in hyperparameters['lr']:
                models = [GraphSAGE(in_channels=dataset.num_features, hidden_channels=256, number_of_classes=dataset.num_classes, num_of_hidden_layers=4, device=device)]
                for model in models:
                    print(f'VC : {vc}, temp : {temperature:.5f}, constrative coef : {constrative_coef:.5f}, lr : {lr:.5f} ')
                    max_loss = 10000
                    early_stopping_count = 0
                    print(f'Model: {model.name} | Number of parameters: {model.get_n_params()}')
                    model = model.to(device)
                    data_for_tuning = data_for_tuning.to(device)
                    criterion = nn.CrossEntropyLoss()
                    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
                    losses = []
                    accs = []
                    val_losses = []
                    val_accs = []
                    for epoch in range(5000):
                        loss, acc = train_constrative_model(model, data_for_tuning, optimizer, criterion, cnode_weight=2,
                                                            constrative_coef=constrative_coef, temperature=temperature, positive_sampling=True)
                        losses.append(loss.item())
                        accs.append(100*acc)
                        val_loss, val_acc = valid_model(model, data_for_tuning, criterion, cnode_weight=2,
                                                        constrative_coef=constrative_coef, temperature=temperature, positive_sampling=True)
                        val_accs.append(100*val_acc)
                        if val_loss < max_loss:
                            max_loss = val_loss
                            early_stopping_count = 0
                        else:
                            early_stopping_count += 1
                            if early_stopping_count > EARLY_STOPPING:
                                print("Early stopping..")
                                break
                        if epoch%10==0:
                            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Train Acc: {100*acc:.2f}, Valid Loss: {val_loss:.4f}, Valid Acc: {100*val_acc:.2f}')
                        if epoch > 500:
                            if val_acc < 0.1:
                                print('underfitting...')
                                break
                    report = test_model(model, data_for_tuning)
                    result = pd.DataFrame(report).T
                    result_sliced = result.iloc[:-3 if len(result) < 23 else 20, :]
                    test_acc = result.loc['accuracy'][0]
                    result.loc['minorities-f1',:] = result_sliced.mean(axis=0)
                    result.to_csv(os.path.join(SAVE_PATH, f'{model.name}_layers{model.num_of_hidden_layers}_neurons{model.hidden_channels}'+'.csv'))
                    result = model(data_for_tuning.x.to(device), data_for_tuning.edge_index.to(device))[0].cpu()
                    global_mad = mad(result).item()
                    mad_gap = madgap(result, data_for_tuning.edge_index).item()
                    
                    exp_result_dict = {
                        'model' : model.name,
                        'virtualnode' : vc,
                        'temperature' : temperature,
                        'constrative coef' : constrative_coef,
                        'lr' : lr,
                        'train_acc' : acc,
                        'train_loss' : loss,
                        'val_acc' : val_acc,
                        'val_loss' : val_loss,
                        'test_acc' : test_acc,
                        'macro f1' : pd.DataFrame(report).T.loc['macro avg', 'f1-score'],
                        'micro f1' : pd.DataFrame(report).T.loc['weighted avg', 'f1-score'],
                        'minor f1' : pd.DataFrame(report).T[:-3].sort_values(by='support', ascending=False)[-11:].mean()['f1-score'],
                        'mad' : global_mad,
                        'madgap' : mad_gap                            
                    }
                    
                    tuning_result = tuning_result.append(exp_result_dict, ignore_index=True)
                    
                    print(f'global_mad: {global_mad}')
                    print(f'madgap: {mad_gap}')
                    print(f'Test Acc: {100*test_acc}')
                    
                    print('==========================================', end='\n\n')
                    del model
                    torch.cuda.empty_cache()   
                    
    del data_for_tuning     
    torch.cuda.empty_cache()        


VC : UnidirectionalVirtualClassNode(), temp : 0.10000, constrative coef : 0.00010, lr : 0.00010 
Model: GraphSAGE | Number of parameters: 4853760
Epoch: 000, Train Loss: 8.5466, Train Acc: 1.00, Valid Loss: 8.5662, Valid Acc: 0.79
Epoch: 010, Train Loss: 8.5278, Train Acc: 3.57, Valid Loss: 8.5501, Valid Acc: 3.92
Epoch: 020, Train Loss: 8.5068, Train Acc: 4.07, Valid Loss: 8.5387, Valid Acc: 4.64
Epoch: 030, Train Loss: 8.4984, Train Acc: 4.65, Valid Loss: 8.5324, Valid Acc: 4.64
Epoch: 040, Train Loss: 8.4842, Train Acc: 4.56, Valid Loss: 8.5284, Valid Acc: 4.64
Epoch: 050, Train Loss: 8.4786, Train Acc: 4.52, Valid Loss: 8.5225, Valid Acc: 4.64
Epoch: 060, Train Loss: 8.4737, Train Acc: 4.59, Valid Loss: 8.5099, Valid Acc: 4.64
Epoch: 070, Train Loss: 8.4458, Train Acc: 4.86, Valid Loss: 8.4857, Valid Acc: 4.64
Epoch: 080, Train Loss: 8.4262, Train Acc: 4.99, Valid Loss: 8.4504, Valid Acc: 4.64
Epoch: 090, Train Loss: 8.4085, Train Acc: 4.68, Valid Loss: 8.4262, Valid Acc: 4.64
Epoc

In [8]:
tuning_result.to_csv('tuning result', index=False)